In [19]:
%load_ext autoreload
%autoreload 2
from model import *
from neurons import *
from synaptics import *
from utils import *
from protocols import *
from shallow_models import *
from vis_functions import *
from pyvis.network import Network
res = .1
plt.style.use(['dark_background'])


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
snn = net_v0()
snn.show_config()

0
1
2
3
4
5


{'Neurons': {0: <neurons.Spikes_at_will at 0x151283cecd0>,
  1: <neurons.Spikes_at_will at 0x151283e9410>,
  2: <neurons.Spikes_at_will at 0x151283eab50>,
  3: <neurons.Spikes_at_will at 0x151283e9e90>,
  4: <neurons.Spikes_at_will at 0x151283ea1d0>,
  5: <neurons.Izhikevich at 0x151283e9a10>},
 'Synapses': {(0, 5): <synaptics.Synapse at 0x15127d32350>,
  (1, 5): <synaptics.Synapse at 0x15127aa8890>,
  (2, 5): <synaptics.Synapse at 0x151284cff90>,
  (3, 5): <synaptics.Synapse at 0x151284cf750>,
  (4, 5): <synaptics.Synapse at 0x151284cd2d0>}}

In [24]:
gatherer = Gatherer(snn, list(range(5)), [5])
gatherer.gather_stats()
gatherer.gather_stats()
gatherer.get_stats()

KeyError: (0, 1, 2, 3, 4)

In [ ]:
snn.reload_graph()
for n1,n2,attr in snn.graph.edges(data=True):
    print (n1,n2,attr)
pos = nx.spring_layout(snn.graph)
nx.draw(snn.graph, node_size=40)
labels = nx.get_edge_attributes(snn.graph, 'weight')


In [ ]:
G = snn.graph
pos = nx.spring_layout(G)
nx.draw(G, pos)
nx.draw_networkx_edge_labels(G, pos)
plt.show()